In [30]:
import requests
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import boto3
from datetime import datetime
import os
from io import BytesIO

In [31]:
link2 = f"https://api.openweathermap.org/data/2.5/weather?q=novo hamburgo,br&APPID=550d57361bde0d30b0dd389950c781ad"

In [32]:
link2

'https://api.openweathermap.org/data/2.5/weather?q=novo hamburgo,br&APPID=550d57361bde0d30b0dd389950c781ad'

In [33]:
requisicao = requests.get(link2)

In [34]:
requisicao.json()

{'coord': {'lon': -51.1306, 'lat': -29.6783},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01n'}],
 'base': 'stations',
 'main': {'temp': 291.34,
  'feels_like': 291.56,
  'temp_min': 290.23,
  'temp_max': 292.63,
  'pressure': 1012,
  'humidity': 90,
  'sea_level': 1012,
  'grnd_level': 1008},
 'visibility': 10000,
 'wind': {'speed': 0.93, 'deg': 306, 'gust': 1.41},
 'clouds': {'all': 2},
 'dt': 1700368084,
 'sys': {'type': 2,
  'id': 2020383,
  'country': 'BR',
  'sunrise': 1700381970,
  'sunset': 1700431230},
 'timezone': -10800,
 'id': 3456068,
 'name': 'Novo Hamburgo',
 'cod': 200}

In [35]:
df = requisicao.json()

In [36]:
df1 = pd.json_normalize(df)

In [37]:
df_lon_lat = pd.DataFrame(df['coord'], index=[0])

In [38]:
df_lon_lat

,lon,lat
0,-51.1306,-29.6783


In [39]:
df_weather = pd.DataFrame(df['weather'], index=[0])

In [40]:
df_weather

,id,main,description,icon
0,800,Clear,clear sky,01n


In [41]:
df_main = pd.DataFrame(df['main'], index=[0])

In [42]:
df_main

,temp,feels_like,temp_min,temp_max,pressure,humidity,sea_level,grnd_level
0,291.34,291.56,290.23,292.63,1012,90,1012,1008


In [43]:
df1 = pd.DataFrame(df1)

In [44]:
df1

,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,...,main.grnd_level,wind.speed,wind.deg,wind.gust,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000,1700368084,-10800,3456068,Novo Hamburgo,200,-51.1306,-29.6783,...,1008,0.93,306,1.41,2,2,2020383,BR,1700381970,1700431230


In [45]:
selected_columns = [
    "base",
    "visibility",
    "dt",
    "timezone",
    "id",
    "name",
    "cod",
    "coord.lon",
    "coord.lat",
    "main.temp",
    "main.feels_like",
    "main.temp_min",
    "main.temp_max",
    "main.pressure",
    "main.humidity",
    "main.sea_level",
    "main.grnd_level",
    "wind.speed",
    "wind.deg",
    "wind.gust",
    "clouds.all",
    "sys.type",
    "sys.id",
    "sys.country",
    "sys.sunrise",
    "sys.sunset",
]

In [46]:
df_clumns_selected = df1[selected_columns]

In [47]:
df_clumns_selected

,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,main.temp,...,main.grnd_level,wind.speed,wind.deg,wind.gust,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,stations,10000,1700368084,-10800,3456068,Novo Hamburgo,200,-51.1306,-29.6783,291.34,...,1008,0.93,306,1.41,2,2,2020383,BR,1700381970,1700431230


In [48]:
super_df = pd.concat([df_weather, df_lon_lat, df_main, df_clumns_selected], axis=0, ignore_index=True)

In [49]:
super_df.to_parquet("C:/Users/crist/Desktop/s3_testes/super_df.parquet")

In [50]:
super_df

,id,main,description,icon,lon,lat,temp,feels_like,temp_min,temp_max,...,main.grnd_level,wind.speed,wind.deg,wind.gust,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,800.0,Clear,clear sky,01n,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,-51.1306,-29.6783,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,291.34,291.56,290.23,292.63,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3456068.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1008.0,0.93,306.0,1.41,2.0,2.0,2020383.0,BR,1.700382e+09,1.700431e+09


putaria com o boto3

In [51]:
# Defina suas credenciais explicitamente (não recomendado)
profile_name = 'CRISTIAN_AWS'

s3 = boto3.session.Session(profile_name=profile_name).client('s3')

In [52]:
bucket_name = 'cristian-teste-buckt-from-python-code'

In [53]:
local_file_path = "C:/Users/crist/Desktop/s3_testes/super_df.parquet"

In [54]:
dia_da_exucucao = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')

In [55]:
dia_da_exucucao

'2023-11-19-01-28-04'

In [56]:
s3_file_name = f'df_teste_{dia_da_exucucao}.parquet'

In [57]:
s3_file_name

'df_teste_2023-11-19-01-28-04.parquet'

In [58]:
s3.upload_file(local_file_path, bucket_name, s3_file_name)

---------------------------------

CATUABA